# IMPORTS 

In [10]:
import pandas as pd
pd.set_option("max_colwidth", None)

import pycaret
import numpy as np
import matplotlib.pyplot as plt
from pycaret.classification import * 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from functions.homebrew import *

# LOAD DATA

In [11]:
df = pd.read_csv('./data/df_complex.csv').drop('Unnamed: 0', axis=1)

In [12]:
train.head()

zipconvert2 zipconvert3 zipconvert4 zipconvert5 homeowner  num_child  \
0          No         Yes          No          No        No          1   
1         Yes          No          No          No       Yes          1   
2          No         Yes          No          No       Yes          1   
3          No          No          No         Yes       Yes          1   
4          No          No          No         Yes        No          1   

   income female  wealth  home_value  ...  target  interaction_wealth_income  \
0       1    Yes       8         316  ...   Donor                          8   
1       1     No       5        1077  ...   Donor                          5   
2       4     No       8         944  ...   Donor                         32   
3       4    Yes       0         576  ...   Donor                          0   
4       4    Yes       8         564  ...   Donor                         32   

   income_to_home_ratio  wealth_to_home_ratio  gift_to_income_ratio  \
0              0.003165              0.025316                 65.00   
1              0.000929              0.004643                292.00   
2              0.004237              0.008475                  9.25   
3              0.006944              0.000000                 34.25   
4              0.007092              0.014184                 20.50   

   average_gift_size_to_largest_gift_ratio  \
0                                 0.393939   
1                                 0.811111   
2                                 0.840909   
3                                 0.537255   
4                                 0.683333   

   total_gifts_to_number_of_promotions  homeowner_children        zip_gender  \
0                             1.031746                No_1  No_Yes_No_No_Yes   
1                             3.518072               Yes_1   Yes_No_No_No_No   
2                             1.480000               Yes_1   No_Yes_No_No_No   
3                             1.957143               Yes_1  No_No_No_Yes_Yes   
4                             2.342857                No_1  No_No_No_Yes_Yes   

   time_since_last_gift_bucket  
0                          24+  
1                        13-24  
2                          24+  
3                          24+  
4                          24+  

[5 rows x 30 columns]

In [13]:
train = df[df['type'] == 'train'].drop('type',axis =1)
dev = df[df['type'] == 'dev'].drop('type',axis =1)
test = df[df['type'] == 'test'].drop('type',axis =1)

# VIF

In [18]:
train = train.dropna()

In [19]:

dummies = pd.get_dummies(train, drop_first=True)

kept, removed = remove_high_vif_features(X=dummies.drop('target_No Donor', axis=1), y=dummies['target_No Donor'], vif_threshold=10)
print('REMOVED:', removed)


new_train = dummies.drop(columns=removed)
new_dev = pd.get_dummies(dev, drop_first=True).drop(columns=removed)
new_test = pd.get_dummies(test, drop_first=True).drop(columns=removed)

MissingDataError: exog contains inf or nans

In [22]:
new_train['target_No Donor'].value_counts()

1    1213
0    1187
Name: target_No Donor, dtype: int64

# ATTEMPT WITH PYCARET

In [23]:
from pycaret.classification import setup, compare_models

# Initialize and setup the experiment
exp = setup(data=new_train, target='target_No Donor', session_id=27, fix_imbalance=True)

# Compare models to find the best one
best = compare_models()

Processing:   0%|          | 0/61 [00:00<?, ?it/s]

# DEV ACCURACTY

In [41]:
preds = predict_model(best, data=new_dev.drop(columns='target_No Donor'))
preds = preds['prediction_label']
actual = new_dev['target_No Donor']
accuracy_score(preds, actual)

0.5366666666666666

# FINE TUNING

In [50]:
tuned_best = tune_model(best, optimize='Accuracy')

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [51]:
preds = predict_model(tuned_best, data=new_dev.drop(columns='target_No Donor'))
preds = preds['prediction_label']
actual = new_dev['target_No Donor']
accuracy_score(preds, actual)

0.5316666666666666